# Notes 3


In [1]:
import duckdb
import os
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from graphviz import Digraph
from sympy.physics.units import magnetic_density

from variables import Variables as vars

plt.style.use('../../notebook.mplstyle')

os.makedirs("./output", exist_ok=True)

In [2]:
# con.close()
con = duckdb.connect("./data/us_births.db") #, read_only=True)

In [10]:
meduc_df = con.execute(
    """
    SELECT
        b.year,
        CASE WHEN b.mage_c < 35 THEN '<35' ELSE '>=35' END as mage_group,
        b.meduc,
        COUNT(*) as birth_count,
        SUM(b.down_ind)::INT as ds_recorded,
        SUM(b.p_ds_lb_nt) as ds_est_no_term,
        SUM (b.ds_rec_weight) as case_weighted,
    FROM us_births as b
    GROUP BY b.year, mage_group, b.meduc
    ORDER BY b.year, mage_group, b.meduc
    """
).df()
meduc_df.to_csv(f"./output/meduc_age_group_by_year-{datetime.now().strftime("%Y%m%d%H%M")}.csv")
meduc_df

,year,mage_group,meduc,birth_count,ds_recorded,ds_est_no_term,case_weighted
0,1989,<35,<NA>,3705383,1491,3374.286550,3078.4824
1,1989,>=35,<NA>,340310,463,2061.517526,972.0088
2,1990,<35,<NA>,3794667,1468,3474.508813,3196.5755
3,1990,>=35,<NA>,368250,515,2242.120406,1139.0436
4,1991,<35,<NA>,3730142,1272,3423.876083,3147.3382
...,...,...,...,...,...,...,...
446,2024,>=35,6,219922,240,1500.735940,572.5012
447,2024,>=35,7,142069,119,1005.395800,274.9084
448,2024,>=35,8,53331,33,394.766314,80.9871
449,2024,>=35,9,19695,27,166.382560,68.8783
